<a href="https://colab.research.google.com/github/bngaruiya/CropDiseaseClassifier/blob/master/crop_diseases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#  Mounts Google Colab on Gdrive.
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Sets the location of your kaggle.json file which contains the configuration
# details of your kaggle API. This is where we will get our data(Images) from.

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Crop Diseases/"

In [0]:
%cd /content/gdrive/My Drive/Crop Diseases

/content/gdrive/My Drive/Crop Diseases


In [0]:
# This code downloads the training data from Kaggle. Note that it should only be
# run once. You can comment this line once done so that it does not affect your
# subsequent runs.

# !kaggle datasets download -d emmarex/plantdisease

In [0]:
# This code unzips the compressed file received from kaggle. It should also be run
# once and commented for all other subsequent runs.

# !unzip \*.zip  && rm *.zip

In [0]:
# Pytorch specific libraries
import torch
import torchvision
import torch.nn.functional as F
from torch import nn, optim
from torchvision import datasets, transforms, models
import torch.utils.data
from torch.optim.lr_scheduler import StepLR

# Other useful python libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from collections import OrderedDict
import json

In [0]:
# Checks the device available for training. Whenever possible run your training
# on GPU, it is much faster. If GPU is not available the CPU is set as default.

train_on_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
# I manually split my data into training, validation and testing datasets.
# Training is ~0.8, validation and testing is ~0.1 each. You could also experiment
# with 0.7, 0.2(val), and 0.1(test). Ensure the training dataset is larger.

data_dir = 'PlantVillage'
train_dir = os.path.join(data_dir, 'train/')
val_dir = os.path.join(data_dir, 'val/')
test_dir = os.path.join(data_dir, 'test/')

In [0]:
# Defines transformations which ensures we have a richer dataset to work with.
# Notice that some training transforms are not applied to val and test.

train_transforms = transforms.Compose([transforms.Resize(size=224),
                                      transforms.RandomHorizontalFlip(),
                                        transforms.RandomVerticalFlip(p=0.1),
                                      transforms.RandomRotation(30),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.4548, 0.4758, 0.3215],
                                                          [0.4548, 0.4758, 0.3215])])
valid_transforms = transforms.Compose([transforms.Resize(size=224),
                                        transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.4548, 0.4758, 0.3215],
                                                          [0.4548, 0.4758, 0.3215])])

test_transforms = transforms.Compose([transforms.Resize(size=224),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.4548, 0.4758, 0.3215],
                                                          [0.4548, 0.4758, 0.3215])])

In [0]:
# Create a train, validation and testing dataset.

train_dataset = datasets.ImageFolder(
    train_dir,
    transform = train_transforms
)
val_dataset = datasets.ImageFolder(
    val_dir,
    transform = valid_transforms
)
test_dataset = datasets.ImageFolder(
    test_dir,
    transform = test_transforms
)

In [0]:
# The dataset is highly imbalanced which tends to bias classification towards the
# more dominant classes. This sampler will randomly select a balanced dataset from
# the data ensuring that it is balanced and thus not biased.

class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
        callback_get_label func: a callback-like function which takes two arguments - dataset and index
    """

    def __init__(self, dataset, indices=None, num_samples=None, callback_get_label=None):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = list(range(len(dataset))) \
            if indices is None else indices

        # define custom callback
        self.callback_get_label = callback_get_label

        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices) \
            if num_samples is None else num_samples
            
        # distribution of classes in the dataset 
        label_to_count = {}
        for idx in self.indices:
            label = self._get_label(dataset, idx)
            if label in label_to_count:
                label_to_count[label] += 1
            else:
                label_to_count[label] = 1
                
        # weight for each sample
        weights = [1.0 / label_to_count[self._get_label(dataset, idx)]
                   for idx in self.indices]
        self.weights = torch.DoubleTensor(weights)

    def _get_label(self, dataset, idx):
        if isinstance(dataset, torchvision.datasets.MNIST):
            return dataset.train_labels[idx].item()
        elif isinstance(dataset, torchvision.datasets.ImageFolder):
            return dataset.imgs[idx][1]
        elif isinstance(dataset, torch.utils.data.Subset):
            return dataset.dataset.imgs[idx][1]
        elif self.callback_get_label:
            return self.callback_get_label(dataset, idx)
        else:
            raise NotImplementedError
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

In [0]:
# sets a batch size to work with. You could also try 16, 32, 64, 256.

batch_size = 128

In [0]:
# Create dataloaders from the datasets above

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler=ImbalancedDatasetSampler(train_dataset))
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [0]:
# retrieves a list of our classes to predict from the dataset
classes = os.listdir(train_dir)
print(classes)

# Create a dictionary of our classes mapped to their index values
train_dataset.class_to_idx
idx_to_class = {j:i for i, j in train_dataset.class_to_idx.items()}

# Creates a json file of our classes
with open('classes.json', 'w') as f:
    json.dump(idx_to_class, f)

['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato_healthy']


In [0]:
# We will be implementing transfer learning using a pretrained resnet model.
# You could try other models as well and check their accuracy

model_transfer = models.resnet152(pretrained=True)

# Freeze parameters so we don't backprop through them
for param in model_transfer.parameters():
    param.requires_grad = False

# Creating the classifier ordered dictionary first
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(2048, 512)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(512, len(classes))),
                          ]))

# Replacing the pretrained model classifier with our classifier
model_transfer.fc = classifier

# Checks if GPU is available and moves the model there.
if train_on_gpu:
    model_transfer.cuda()

# Checks the model architecture after modification of the last layer
print(model_transfer)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth



ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1,

In [0]:
# Sets the loss function for training
criterion_transfer = nn.CrossEntropyLoss()

# sets the Optimization function
optimizer_transfer = optim.Adam(model_transfer.fc.parameters(), lr = 0.0001)

# sets a linear learning rate decay function
scheduler_transfer = StepLR(optimizer_transfer, step_size=5, gamma=0.3)

In [0]:
# Defines the training and validation functions
def train (epochs, loaders, model, optimizer, scheduler, criterion, train_on_gpu, save_path):
    valid_loss_min = np.Inf

    for epoch in range(1, epochs+1):
        # keep track of the training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        scheduler=scheduler

        #####################
        ## train the model ##
        #####################
        print('Epoch:', epoch,'LR:', scheduler.get_last_lr())
        for batch_i, (data, target) in enumerate(loaders['train']):
            ## Move data and target to GPU
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            ## Zero the gradients
            optimizer.zero_grad()
            ## Forward pass
            output = model(data)
            ## Calculate the loss
            loss = criterion(output, target)
            ## backpropagation
            loss.backward()
            ## Single step optimization
            optimizer.step()
            ## Update the training loss
            train_loss += ((1 / (batch_i + 1)) * (loss.data - train_loss))

            if batch_i % 5 == 4 : #Print training loss for every 128 batches
                print('Epoch %d, Batch %d, loss %.16f' %(epoch, batch_i + 1, train_loss / 5))
                train_loss = 0.0

        ######################
        # Validate the model #
        ######################

        model.eval()
        for batch_i, (data, target) in enumerate(loaders['valid']):
            # move data and target to GPU
            if train_on_gpu:
                data, target = data.cuda(), target.cuda()
            with torch.no_grad():
                output = model(data)
            loss = criterion(output, target)
            valid_loss += ((1 / (batch_i + 1)) * (loss.data - valid_loss))
            
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model...'.format(valid_loss_min, valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
        scheduler.step()
            
    return model

In [0]:
## Define the loaders
loaders_transfer = {
    "train": train_loader,
    "valid": val_loader,
    "testing": test_loader
}

In [0]:
# Training and Validation
model_transfer = train(20, loaders_transfer, model_transfer, optimizer_transfer, scheduler_transfer, criterion_transfer, train_on_gpu, 'model_transfer.pt')

Epoch: 1 LR: [0.0001]
Epoch 1, Batch 5, loss 0.5294721722602844
Epoch 1, Batch 10, loss 0.2549911737442017
Epoch 1, Batch 15, loss 0.1640819013118744
Epoch 1, Batch 20, loss 0.1175818592309952
Epoch 1, Batch 25, loss 0.0910038873553276
Epoch 1, Batch 30, loss 0.0718920156359673
Epoch 1, Batch 35, loss 0.0586107373237610
Epoch 1, Batch 40, loss 0.0493436418473721
Epoch 1, Batch 45, loss 0.0415143668651581
Epoch 1, Batch 50, loss 0.0356717780232430
Epoch 1, Batch 55, loss 0.0307558458298445
Epoch 1, Batch 60, loss 0.0272042397409678
Epoch 1, Batch 65, loss 0.0238702725619078
Epoch 1, Batch 70, loss 0.0208853576332331
Epoch 1, Batch 75, loss 0.0185649897903204
Epoch 1, Batch 80, loss 0.0168421696871519
Epoch 1, Batch 85, loss 0.0150096938014030
Epoch 1, Batch 90, loss 0.0139757888391614
Epoch 1, Batch 95, loss 0.0127636929973960
Epoch 1, Batch 100, loss 0.0117804035544395
Epoch 1, Batch 105, loss 0.0112321227788925
Epoch 1, Batch 110, loss 0.0098556010052562
Epoch 1, Batch 115, loss 0.008

In [0]:
# Create a models folder to save the trained model dictionary
!mkdir -p '/content/gdrive/My Drive/Crop Diseases/models/'

# Copy the saved model dictionary to Gdrive
!cp model_transfer.pt '/content/gdrive/My Drive/Crop Diseases/models/'

In [0]:
# Loads the model dictionary for inference
model_transfer.load_state_dict(torch.load('model_transfer.pt'), ) 

<All keys matched successfully>

In [0]:
# Defines the model testing function
def test(loaders, model, criterion, use_cuda):
    # Monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    
    model.eval()
    for batch_i, (data, target) in enumerate(loaders['testing']):
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
    
test(loaders_transfer, model_transfer, criterion_transfer, train_on_gpu)

Test Loss: 8.196065


Test Accuracy: 91% (2909/3190)


In [0]:
# Defines the prediction function. This will receive an image, and return the
# predicted class of the image

def predict_crop_disease(img_path):
    # load the image and return the predicted breed
    img = Image.open(img_path).convert('RGB')
    transform = transforms.Compose([transforms.Resize(size=224),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.4548, 0.4758, 0.3215],
                                                          [0.4548, 0.4758, 0.3215])])
    img_tensor = transform(img).unsqueeze(0)
    if train_on_gpu:
        img_tensor = img_tensor.cuda()
    # Turn on evaluation mode
    model_transfer.eval()
    # Get the predicted class
    with torch.no_grad():
        output = model_transfer(img_tensor)
        pred = torch.argmax(output).item()
        # target = torch.max(target.data)

    #Turn off evaluation mode
    # model_transfer.train()
    # Get the predicted class using the defined model
    crop_disease_class = classes[pred]
    return crop_disease_class

In [0]:
# def run(path):
#     for img in (path):
#         img = Image.open(path)
#         plt.show(img)
#         plt.show()
#         print(f'The crop disease in this photo is ... {predict_crop_disease(path)}')  
#         print('################################################')

In [0]:
# test_dir = '/content/gdrive/My Drive/Crop Diseases/predict/'
# for file in os.listdir(test_dir):
#     run(test_dir + file)